In [1]:
import pandas as pd
import numpy as np
import glob
import os
import itertools
from itertools import permutations 
from tqdm import tqdm
from collections import defaultdict, Counter
from copy import deepcopy
import gc

In [2]:
import pyarrow
import pyarrow.parquet as pq

In [3]:
INPUT_PARQUET_PATH = "/home/gmoreira/dataset/ecommerce_preproc_2019-*/ecommerce_preproc.parquet/session_start_date=*"
OUTPUT_NEG_SAMPLES_PARQUET_PATH = "/home/gmoreira/dataset/neg_samples.parquet"

In [4]:
UNIFORM_SAMPLING = 'uniform'
RECENCY_SAMPLING = 'recency'
RECENT_POPULARITY_SAMPLING = 'popularity'
COOCURRENCE_SAMPLING = 'cooccurrence'

In [5]:
NEGATIVE_SAMPLING_STRATEGY = COOCURRENCE_SAMPLING

In [6]:
BATCH_SIZE = 1000
BATCHES_TO_UPDATE_ITEM_STATS = 3
BATCHES_TO_APPEND_ROWS_WITH_NEG_SAMPLES = 5
ITEM_STATS_KEEP_LAST_N_DAYS = 1.0
SEQUENCE_LENGTH = 20
NUM_NEG_SAMPLES = 50

In [7]:
input_parquet_files = sorted(glob.glob(INPUT_PARQUET_PATH+'*'))
input_parquet_files

['/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-01',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-02',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-03',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-04',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-05',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-06',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-07',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-08',
 '/home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-09',
 '/home/gmoreira/da

In [8]:
def get_output_path_parquet_neg_samples(input_parquet_filename):
    return input_parquet_filename \
        .replace('ecommerce_preproc.parquet', 'ecommerce_preproc_neg_samples_{}_strategy_{}.parquet' \
                     .format(NUM_NEG_SAMPLES, NEGATIVE_SAMPLING_STRATEGY)) + '.parquet'

In [58]:
'''
def get_files(data_paths):
    paths = [['file://' + p for p in glob.glob(path + "/*.parquet")] for path in data_paths]
    return list(itertools.chain.from_iterable(paths))
input_parquet_files = get_files([INPUT_PARQUET_PATH])
'''
pass

In [10]:
'''
#Works but cannot be used now because the preprocessed sessions are not sorted by timestamp
def read_parquet_generator(filenames, batch_size=128):
    for filename in filenames:
        for batch in pq.read_table(filename).to_batches(batch_size):
            yield batch.to_pandas()
            
parquet_reader = read_parquet_generator([INPUT_PARQUET_PATH], batch_size=BATCH_SIZE)            
'''
pass

In [11]:
def split_dataframe_into_chuncks_generator(df, chunk_size): 
    number_chunks = len(df) // chunk_size + 1
    for i in range(number_chunks):
        yield df[i*chunk_size:(i+1)*chunk_size]

In [12]:
def insert_update_session_items_metadata(row):
    #Uses the session start as the event timestamp (as the sess_etime_seq might sometimes be many days before because of outlier sessions with more than 120 min duration (< 1%))
    etime = row['session_start_ts']
    #For each session
    for pid, csid, ccid, bid, price, relative_price, prod_recency in zip(
                                                        #row['sess_etime_seq'],
                                                        row['sess_pid_seq'], 
                                                        row['sess_csid_seq'],
                                                        row['sess_ccid_seq'],
                                                        row['sess_bid_seq'],
                                                        row['sess_price_seq'],
                                                        row['sess_relative_price_to_avg_category_seq'],
                                                        row['sess_product_recency_seq']):

        #If this item was not processed before
        if pid != 0:
            if pid in items_df.index:
                curr_row = items_df.loc[pid]

                first_ts = curr_row['first_ts']
                last_ts = curr_row['last_ts']
                if etime > last_ts:
                    last_ts = etime
            else:
                first_ts = etime
                last_ts = etime

            #Including or updating the item metadata
            items_df.loc[pid] = pd.Series({'csid': csid,
                                           'ccid': ccid,
                                           'bid': bid,
                                           'price': price,
                                           'relative_price_to_avg_category': relative_price,
                                           'product_recency': prod_recency,
                                           'first_ts': first_ts,
                                           'last_ts': last_ts})

In [13]:
session_cooccurrences_log_list = []

def append_session_coocurrences_log(row):
    global session_cooccurrences_log_list
    min_ts = min([t for t in row['sess_etime_seq'] if t != 0])
    valid_pids = list(set(list([p for p in row['sess_pid_seq'] if p != 0])))
    
    if len(valid_pids) > 1:
        items_permutations = permutations(valid_pids, 2)        
        new_coo_df = pd.DataFrame(items_permutations, columns=['pid_a', 'pid_b'])
        new_coo_df['ts'] = min_ts
        #This flag is used for counting unique values from this table to compute popularity
        new_coo_df['count_flag'] = ([1] + [0]*(len(valid_pids)-2))*len(valid_pids)
        
        session_cooccurrences_log_list.append(new_coo_df)

def concat_sessions_coocurrences_log():
    global items_coocurrence_df, session_cooccurrences_log_list
    items_coocurrence_df = pd.concat([items_coocurrence_df] + session_cooccurrences_log_list)
    session_cooccurrences_log_list = []

In [14]:
def remove_old_interactions(keep_last_n_days):
    global items_coocurrence_df
    last_ts = items_coocurrence_df['ts'].max()
    keep_last_n_secs = keep_last_n_days * 24 * 60 * 60
    items_coocurrence_df = items_coocurrence_df[items_coocurrence_df['ts'] >= (last_ts - keep_last_n_secs)]

In [15]:
def update_items_temporal_relevance_decay():    
    global items_temporal_relev_df
    max_reference_ts = items_df['first_ts'].max()
    prods_days_age = (max_reference_ts - items_df['first_ts']) / (60 * 60 * 24)

    time_relev_by_item_series = prod_relevance_decay(prods_days_age)
    items_temporal_relev_df = time_relev_by_item_series / time_relev_by_item_series.sum()

In [16]:
def update_items_coocurrences_counts():
    global items_coocurence_counts_df
    items_coocurence_counts_df = items_coocurrence_df.groupby(['pid_a','pid_b']).size().to_frame('count') \
                                    .reset_index(level=[1])

In [17]:
def update_items_recent_popularity():
    global items_recent_pop_df
    items_recent_pop_df = items_coocurrence_df[items_coocurrence_df['count_flag'] == True] \
            .groupby(['pid_a']).size().to_frame('count')
    items_recent_pop_df['prob'] = items_recent_pop_df['count'] / items_recent_pop_df['count'].sum()

In [18]:
# (83% of relevance in one quarter, 70% in one semester, 50% in one year and 23% in two years)
DAYS_DECAY_FACTOR = 0.002

def prod_relevance_decay(days_age):
    return np.exp(-days_age*DAYS_DECAY_FACTOR)

In [19]:
# (83% of relevance in one quarter, 70% in one semester, 50% in one year and 23% in two years)
DAYS_DECAY_FACTOR = 0.002
# Simulating 2 year of decay on relevance of a product 
for i in np.arange(0,365*2,30):    
    print(i, prod_relevance_decay(i))

0 1.0
30 0.9417645335842487
60 0.8869204367171575
90 0.835270211411272
120 0.7866278610665535
150 0.7408182206817179
180 0.697676326071031
210 0.6570468198150567
240 0.6187833918061408
270 0.5827482523739896
300 0.5488116360940264
330 0.5168513344916992
360 0.4867522559599717
390 0.4584060113052235
420 0.43171052342907973
450 0.4065696597405991
480 0.38289288597511206
510 0.3605949401730783
540 0.3395955256449391
570 0.31981902181630384
600 0.30119421191220214
630 0.2836540264997704
660 0.26713530196585034
690 0.25157855305975646
720 0.23692775868212176


In [38]:
items_df = None
items_coocurrence_df = None
items_coocurence_counts_df = None
items_recent_pop_df = None
items_temporal_relev_df = None

def reset_item_logs_and_statistics():
    global items_df, items_coocurrence_df, items_coocurence_counts_df, items_recent_pop_df, items_temporal_relev_df
    
    items_df = pd.DataFrame(columns={'pid': np.int64,
                                 'csid': np.int32,
                                 'ccid': np.int32,
                                 'bid': np.int32,
                                 'price': np.float,
                                 'relative_price_to_avg_category': np.float,
                                 'product_recency': np.float,
                                 'first_ts': np.int,
                                 'last_ts': np.int
                                }).set_index('pid')
    
    items_coocurrence_df = pd.DataFrame(columns={'pid_a': np.int64, 
                                                 'pid_b': np.int64, 
                                                 'ts': np.int32, 
                                                 'count_flag': np.int16})
    
    items_coocurence_counts_df = None
    items_recent_pop_df = None
    items_temporal_relev_df = None

In [47]:
items_coocurence_counts_df.loc[2].sort_values('count', ascending=False)

,pid_b,count
pid_a,,
2,7,1106
2,4,1091
2,36,760
2,13,624
2,6,577
...,...,...
2,9663,1
2,9684,1
2,9692,1


In [21]:
def get_uniform_sampling_item_ids(n_samples):
    return np.random.choice(items_df.index, min(n_samples, len(items_df)), replace=False)

In [22]:
def get_popularity_sampling_item_ids(n_samples):
    return np.random.choice(items_recent_pop_df.index, min(n_samples, len(items_recent_pop_df)), replace=False, 
                            p=items_recent_pop_df['prob']).tolist()

In [40]:
def get_coocurrence_sampling_item_ids(pid, n_samples):
    samples = []
    if pid in items_coocurence_counts_df.index:
        coocurrent_df = items_coocurence_counts_df.loc[pid]
        #Dealing with cases when there is only one co-occurrent item (loc() returns a Series)
        if type(coocurrent_df) is pd.Series:
            coocurrent_df = coocurrent_df.to_frame().T
        coocurrent_df['probs'] = coocurrent_df['count'] / coocurrent_df['count'].sum()
        samples = np.random.choice(coocurrent_df['pid_b'], min(n_samples, len(coocurrent_df)), replace=False, 
                                   p=coocurrent_df['probs']).tolist()
    return samples

In [24]:
def get_recency_sampling_item_ids(n_samples):
    samples = np.random.choice(items_temporal_relev_df.index, min(n_samples, len(items_temporal_relev_df)), replace=False, 
                               p=items_temporal_relev_df.values).tolist()
    return samples

In [25]:
def get_candidate_samples_item_ids(pid, n_samples, strategy, ignore_list=None):
    #To ensure that after removing sessions from the current session we have the required number of samples
    SAMPLES_MULITPLIER = 2
    if strategy == UNIFORM_SAMPLING:
        samples = get_uniform_sampling_item_ids(n_samples*SAMPLES_MULITPLIER)
    elif strategy == RECENCY_SAMPLING:
        samples = get_recency_sampling_item_ids(n_samples*SAMPLES_MULITPLIER)
    elif strategy == RECENT_POPULARITY_SAMPLING:
        samples = get_popularity_sampling_item_ids(n_samples*SAMPLES_MULITPLIER)    
    elif strategy == COOCURRENCE_SAMPLING:
        samples = get_coocurrence_sampling_item_ids(pid, n_samples*SAMPLES_MULITPLIER)
    
        #Completing the list of samples based on global popularity
        if len(samples) < n_samples:
            samples += get_popularity_sampling_item_ids(n_samples - len(samples))
    else:
        raise Exception('Not a valid strategy. Should be: (uniform|recency|popularity|cooccurrence)')
        
    #Removing repeated entries
    samples = list(set(samples))
    #Removing samples from the ignore list
    if ignore_list is not None:
        samples = list([i for i in samples if i not in ignore_list])

    return samples[:n_samples]

In [26]:
#get_candidate_samples_item_ids(63246, 10, strategy=COOCURRENCE_SAMPLING, ignore_list=[2010])

In [27]:
def get_features_for_item_ids(pids):
    return items_df.loc[pids][['csid', 'ccid', 'bid', 'price', 'relative_price_to_avg_category', 'product_recency']] #\
    .to_dict(orient='list')

In [28]:
##%%time
#l= np.random.choice(items_df.index, 50)
#l= np.ones(50, dtype='int')*10
#l = [10] * 50
#l = [33284, 6, 21, 2588, 23069, 1570, 8230, 552, 50, 4152, 55864, 24636, 113213, 65601, 9283, 1097, 1104, 90, 2651, 607, 13920, 7785, 619, 8821, 9337, 4221, 8330, 7307, 8848, 19089, 112784, 2197, 15002, 3234, 3252, 1207, 2745, 40128, 30918, 4299, 61646, 23247, 213, 6871, 8921, 15582, 6367, 15077, 8935, 3820, 1266, 1790, 256, 258, 3844, 1808, 14109, 1311, 8481, 3362, 1318, 2348, 301, 29996, 19245, 1331, 3891, 10553, 830, 1855, 1867, 3415, 2933, 8571, 21371, 390, 903, 1425, 404, 5527, 1958, 23477, 6071, 59321, 7100, 4545, 1479, 28628, 10200, 16857, 3544, 479, 480, 6628, 35812, 2032, 3059, 28153, 2046]
#for i in range(100):
#    for j in range(10):
#        x = get_features_for_item_ids(l)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [29]:
def padarray(A, size):
    if len(A) > size:
        A = A[:size]
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

In [30]:
padarray([1,2,3], 4)

array([1, 2, 3, 0])

In [42]:
def generate_neg_samples(session_pids, user_past_pids, n_samples, strategy):
    neg_samples_dict = defaultdict(list)
    
    #Ignores session items and also recently interacted items
    ignore_ids = set(np.hstack([session_pids, user_past_pids]))
    
    for pid in session_pids:
        if pid != 0:
            #Sampling item idds
            neg_item_ids = get_candidate_samples_item_ids(pid, n_samples, 
                                                          ignore_list=ignore_ids,
                                                         strategy=strategy
                                                         )      
            #Retrieving item features
            neg_item_features_dict = get_features_for_item_ids(neg_item_ids)

            
            pids_padded = padarray(neg_item_ids, n_samples).astype(int)
            neg_samples_dict['sess_neg_pids'].append(pids_padded)
            
            for k, v in neg_item_features_dict.items():
                values = padarray(v, n_samples)
                values = values.astype(int) if k in ['csid', 'ccid', 'bid'] else values.astype(float)
                neg_samples_dict['sess_neg_{}'.format(k)].append(values)
            
        
        else:
            #Creating padding neg samples for each padding interactions
            missing_padding_neg_samples = len(session_pids) - len(neg_samples_dict['sess_neg_pids'])                        
            for k in neg_samples_dict:
                neg_samples = neg_samples_dict[k]
                neg_samples_zeros = np.zeros_like(neg_samples[0])
                for p in range(missing_padding_neg_samples):
                    #Copying shape and dtype from the neg samples of the first interaction
                    neg_samples.append(neg_samples_zeros)
         
    #Concatenating neg. samples of all session interactions because Petastorm data loader 
    #does not support lists of lists. It will require reshaping neg. samples features inside the Pytorch model
    for k in neg_samples_dict:  
        neg_samples_dict[k] = np.hstack(neg_samples_dict[k])        

    return neg_samples_dict

In [32]:
#generate_neg_samples([10,20,30, 0, 0], n_samples=2, strategy='popularity')

In [33]:
def append_new_rows_to_parquet(new_rows_df, path):
    global pq_writer
    new_rows_pa = pyarrow.Table.from_pandas(new_rows_df)
    if pq_writer is None:
        #Creating parent folder recursively
        parent_folder = os.path.dirname(os.path.abspath(path))
        if not os.path.exists(parent_folder):
            os.makedirs(parent_folder)
        #Creating parquet file
        pq_writer = pq.ParquetWriter(path, new_rows_pa.schema) 
    pq_writer.write_table(new_rows_pa)

## Generates neg. samples for all sessions and creates new parquet files

In [44]:
pq_writer = None

reset_item_logs_and_statistics()
try:
    #For each file (day)
    for idx_day, input_file in enumerate(input_parquet_files):
        print('='*40)
        print('[Day {}] Loading sessions from parquet: {}'.format(idx_day, input_file))
        output_filename = get_output_path_parquet_neg_samples(input_file)
        
        if os.path.exists(output_filename):
            raise Exception('Output parquet file already exists')
        
        #Loading parquet file and sorting sessions by timestamp
        sessions_df = pd.read_parquet(input_file)
        sessions_df.sort_values('session_start_ts', inplace=True)
                
        new_rows = []
        
        print('Processing batches')
        #For each batch
        for batch_id, batch in tqdm(enumerate(split_dataframe_into_chuncks_generator(sessions_df, 
                                                                                chunk_size = BATCH_SIZE))):
            print('batch_id', batch_id)            
            #For each row (session)
            for i, row in batch.iterrows():
                insert_update_session_items_metadata(row)
                append_session_coocurrences_log(row)
                
                
                #Ignoring first batch (not computing neg. samples nor saving to parquet)
                if batch_id > 0:   
                    #Generating neg. samples for each interaction in the session
                    session_neg_samples_by_pid_dict = generate_neg_samples(row['sess_pid_seq'], 
                                                                           row['user_pid_seq_bef_sess'],
                                                                           NUM_NEG_SAMPLES, 
                                                                           strategy=NEGATIVE_SAMPLING_STRATEGY
                                                                          )
                    #Merging user and session features with neg samples for the session
                    new_row_with_neg_samples_dict = {**row.to_dict(), **session_neg_samples_by_pid_dict}
                    new_rows.append(new_row_with_neg_samples_dict)
                    
            
            #Each N batches updates item statistics (popularity, recency, co-occurrence)
            #Ps. Do the update for all the first five batches of the first file , for better sampling
            if (batch_id % BATCHES_TO_UPDATE_ITEM_STATS == BATCHES_TO_UPDATE_ITEM_STATS-1) or \
               (idx_day == 0 and batch_id < 5):
                print('[Batch {}] Updating item stats'.format(batch_id))
                remove_old_interactions(ITEM_STATS_KEEP_LAST_N_DAYS)
                if NEGATIVE_SAMPLING_STRATEGY in [RECENT_POPULARITY_SAMPLING, COOCURRENCE_SAMPLING]:
                    concat_sessions_coocurrences_log()
                    update_items_coocurrences_counts()
                    if RECENT_POPULARITY_SAMPLING:
                        update_items_recent_popularity()
                if NEGATIVE_SAMPLING_STRATEGY == RECENCY_SAMPLING:
                    update_items_temporal_relevance_decay()
                
            #Each N batches appends the new rows with neg. samples to parquet file
            if batch_id % BATCHES_TO_APPEND_ROWS_WITH_NEG_SAMPLES == BATCHES_TO_APPEND_ROWS_WITH_NEG_SAMPLES-1: 
                print('[Batch {}] Appending new rows with neg samples to parquet: {}'.format(batch_id,output_filename))
                append_new_rows_to_parquet(pd.DataFrame(new_rows), output_filename)
                del(new_rows)
                new_rows = []
            
               
        #Save pending rows
        if len(new_rows) > 0:
            print('[Batch {}] Appending new rows with neg samples to parquet: {}'.format(batch_id,output_filename))
            append_new_rows_to_parquet(pd.DataFrame(new_rows), output_filename)
            del(new_rows)
            new_rows = []
            
        #Flushing and releasing the current parquet file and proceeding for the new date
        pq_writer.close()
        pq_writer = None
                
        del(sessions_df)
        gc.collect()
finally:
    if pq_writer:
        pq_writer.close()

[Day 0] Loading sessions from parquet: /home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc.parquet/session_start_date=2019-10-01


0it [00:00, ?it/s]

Processing batches
batch_id 0
[Batch 0] Updating item stats


1it [00:09,  9.87s/it]

batch_id 1
[Batch 1] Updating item stats


...

batch_id 59
[Batch 59] Updating item stats
[Batch 59] Appending new rows with neg samples to parquet: /home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc_neg_samples_50_strategy_cooccurrence.parquet/session_start_date=2019-10-01.parquet


60it [29:24, 56.14s/it]

batch_id 60


60it [29:53, 29.88s/it]


KeyboardInterrupt: 

## Loading the parquet with Negative samples with Petastorm

In [48]:
from petastorm.pytorch import DataLoader
from petastorm import make_batch_reader
from petastorm.unischema import UnischemaField
from petastorm.unischema import Unischema
from petastorm.codecs import NdarrayCodec

In [49]:
input_with_neg_parquet_path = 'file:///home/gmoreira/dataset/ecommerce_preproc_2019-10/ecommerce_preproc_neg_samples_50_strategy_cooccurrence.parquet/session_start_date=2019-10-01_full.parquet'

In [50]:
recsys_schema_full = [
  UnischemaField('user_idx', np.int, (), None, True),
#   UnischemaField('user_session', str_, (), None, True),
  UnischemaField('sess_seq_len', np.int, (), None, False),
  UnischemaField('session_start_ts', np.int64, (), None, True),
  UnischemaField('user_seq_length_bef_sess', np.int, (), None, False),
  UnischemaField('user_elapsed_days_bef_sess', np.float, (), None, True),
  UnischemaField('user_elapsed_days_log_bef_sess_norm', np.double, (), None, True),
  UnischemaField('sess_pid_seq', np.int64, (None,), None, True),
  UnischemaField('sess_etime_seq', np.int64, (None,), None, True),
  UnischemaField('sess_etype_seq', np.int, (None,), None, True),
  UnischemaField('sess_csid_seq', np.int, (None,), None, True),
  UnischemaField('sess_ccid_seq', np.int, (None,), None, True),
  UnischemaField('sess_bid_seq', np.int, (None,), None, True),
  UnischemaField('sess_price_seq', np.float, (None,), None, True),
  UnischemaField('sess_dtime_seq', np.float, (None,), None, True),
  UnischemaField('sess_product_recency_seq', np.float, (None,), None, True),
  UnischemaField('sess_relative_price_to_avg_category_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_hour_sin_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_hour_cos_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_month_sin_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_month_cos_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_dayofweek_sin_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_dayofweek_cos_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_dayofmonth_sin_seq', np.float, (None,), None, True),
  UnischemaField('sess_et_dayofmonth_cos_seq', np.float, (None,), None, True),
  UnischemaField('user_pid_seq_bef_sess', np.int64, (None,), None, True),
  UnischemaField('user_etime_seq_bef_sess', np.int64, (None,), None, True),
  UnischemaField('user_etype_seq_bef_sess', np.int, (None,), None, True),
  UnischemaField('user_csid_seq_bef_sess', np.int, (None,), None, True),
  UnischemaField('user_ccid_seq_bef_sess', np.int, (None,), None, True),
  UnischemaField('user_bid_seq_bef_sess', np.int, (None,), None, True),
  UnischemaField('user_price_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_dtime_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_product_recency_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_relative_price_to_avg_category_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_hour_sin_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_hour_cos_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_month_sin_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_month_cos_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_dayofweek_sin_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_dayofweek_cos_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_dayofmonth_sin_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_dayofmonth_cos_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('user_et_dayofmonth_cos_seq_bef_sess', np.float, (None,), None, True),
  UnischemaField('sess_neg_pids', np.int, (None,), None, True),
  UnischemaField('sess_neg_csid', np.int, (None,), None, True),
  UnischemaField('sess_neg_ccid', np.int, (None,), None, True),
  UnischemaField('sess_neg_bid', np.int, (None,), None, True),
  UnischemaField('sess_neg_price', np.float, (None,), None, True),
  UnischemaField('sess_neg_relative_price_to_avg_category', np.float, (None,), None, True),
  UnischemaField('sess_neg_product_recency', np.float, (None,), None, True),
]

In [51]:
with DataLoader(
    make_batch_reader(input_with_neg_parquet_path, 
                num_epochs=1,
                # transform_spec=transform
                schema_fields=recsys_schema_full,
    ), batch_size=2) as train_loader:
    for i, batch in enumerate(train_loader):
        print("i:{}".format(i))
        print(batch)
        print(batch['sess_neg_product_recency'].shape)
        break

/home/gmoreira/anaconda3/envs/transf4rec/lib/python3.7/site-packages/petastorm/unischema.py:197: UserWarning: Can not create dynamic property user_et_dayofmonth_cos_seq_bef_sess because it conflicts with an existing property of Unischema
  'Unischema').format(f.name))
/home/gmoreira/anaconda3/envs/transf4rec/lib/python3.7/site-packages/petastorm/arrow_reader_worker.py:53: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  column_as_pandas = column.data.chunks[0].to_pandas()


i:0
{'user_idx': tensor([550479254, 547827437]), 'sess_seq_len': tensor([21,  3]), 'session_start_ts': tensor([1569921441, 1569921441]), 'user_seq_length_bef_sess': tensor([0, 0]), 'user_elapsed_days_bef_sess': tensor([nan, nan], dtype=torch.float64), 'user_elapsed_days_log_bef_sess_norm': tensor([nan, nan], dtype=torch.float64), 'sess_pid_seq': tensor([[125685,   5713,   3066,  11869,  10670,   4312,  19814,  63599,   8931,
           6909,   7571,   3390,   1697,   3616,  21046,   2666,   7386,   3976,
          23679,  28513],
        [   515,     57,    331,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0]]), 'sess_etime_seq': tensor([[1569921441, 1569921527, 1569921549, 1569921628, 1569921649, 1569921670,
         1569921712, 1569921752, 1569921793, 1569921955, 1569922051, 1569922061,
         1569922096, 1569922146, 1569922168, 1569922273, 1569922283, 1569922301,
         1569

KeyboardInterrupt: 

In [24]:
import pandas as pd
import numpy as np

In [34]:
items_df = pd.DataFrame(columns=[{'pid': np.int32, 'c1': np.int64, 'n1': np.float32}])


TypeError: unhashable type: 'dict'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1652, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'dict'


In [32]:
def df_empty(columns, dtypes, index=None):
    assert len(columns)==len(dtypes)
    df = pd.DataFrame(index=index)
    for c,d in zip(columns, dtypes):
        df[c] = pd.Series(dtype=d)
    return df

items_df = df_empty(columns=['pid', 'c1', 'n1'], dtypes=[np.int32, np.int64, np.float32])

In [90]:
def df_empty(column_dtype_mapping, index=None):    
    df = pd.DataFrame()
    for c in column_dtype_mapping:
        df[c] = pd.Series(dtype=column_dtype_mapping[c])
    return df.set_index(index)

In [156]:
items_df = df_empty({'pid': np.int32, 'c1': np.int64, 'n1': np.float32}, index=['pid'])

In [163]:
items2_df = pd.DataFrame([(1, 10, 12.5)], columns={'pid': np.int32, 'c1': np.int64, 'n1': np.float32}).set_index('pid')

In [149]:
items2_df.loc[lambda df: df['c1'] == 10] = [20, 20.5]

In [166]:
items2_df.dtypes

c1      int64
n1    float64
dtype: object

In [165]:
items2_df.loc[1, ['c1', 'n1']]

c1    10.0
n1    12.5
Name: 1, dtype: float64

In [154]:
items2_df.loc[2] = [20, 15.7]

In [ ]:
items2_df.loc[2] = items2_df[]

In [135]:
items2_df

,c1,n1
pid,,
1,10,12.5


In [126]:
items2_df

,c1,n1
pid,,
1,10.0,12.5
2,20.0,15.7


In [96]:
items_df.loc[0] = pd.Series({'c1': 10, 'n1': 50.5})

In [97]:
items_df

,c1,n1
pid,,
0,10.0,50.5


In [118]:
for i in items2_df.itertuples():
    print(type(i))

<class 'pandas.core.frame.Pandas'>


In [121]:
dir(pd.core.frame)

['ABCDataFrame',
 'ABCIndexClass',
 'ABCMultiIndex',
 'ABCSeries',
 'Any',
 'Appender',
 'Axes',
 'Axis',
 'BlockManager',
 'CachedAccessor',
 'Categorical',
 'DataFrame',
 'DatetimeIndex',
 'DatetimeLikeArray',
 'Dtype',
 'ExtensionArray',
 'FilePathOrBuffer',
 'FrozenSet',
 'Hashable',
 'IO',
 'Index',
 'Iterable',
 'Level',
 'List',
 'NDFrame',
 'Optional',
 'PY37',
 'PeriodIndex',
 'Renamer',
 'Sequence',
 'Series',
 'Set',
 'SparseFrameAccessor',
 'StringIO',
 'Substitution',
 'TYPE_CHECKING',
 'Tuple',
 'Type',
 'Union',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_from_nested_dict',
 '_merge_doc',
 '_numeric_only_doc',
 '_put_str',
 '_shared_doc_kwargs',
 '_shared_docs',
 'abc',
 'algorithms',
 'arrays_to_mgr',
 'cast',
 'cast_scalar_to_array',
 'check_bool_indexer',
 'coerce_to_dtypes',
 'collections',
 'com',
 'console',
 'convert_to_index_sliceable',
 'dedent',
 'deprecate_kwarg',
 'dispatch_fill_zeros',


In [101]:
type(items_df)

pandas.core.frame.DataFrame

In [60]:
type(items_df[0:1])

pandas.core.frame.DataFrame

In [54]:
x = pd.Series({
                                'sess_csid_seq': 100,
                                'sess_price_seq': 270.90,
                                'first_ts': 1594100000, 
                                'last_ts':  1594100000,
                             })

In [58]:
x

sess_csid_seq     1.000000e+02
sess_price_seq    2.709000e+02
first_ts          1.594100e+09
last_ts           1.594100e+09
dtype: float64

In [95]:
items_df

,c1,n1
pid,,


In [94]:
items_df.loc[0] = {'sess_csid_seq': 100,
                    'sess_price_seq': 270.90,
                    'first_ts': 1594100000, 
                    'last_ts':  1594100000,
                 }

ValueError: cannot set a row with mismatched columns

In [87]:
items_df[0:1]

,c1,n1
pid,,
1,12,60.9


In [81]:
items_df

,c1,n1
pid,,
1,12,60.9
0,0 12 dtype: int64,0 60.9 dtype: float64


In [82]:
items_df.dtypes

c1    object
n1    object
dtype: object

In [84]:
pd.Series([12])

0    12
dtype: int64